In [1]:
import numpy as np
import datetime as dt
import tensorflow as tf
from pailab import MLRepo, MeasureConfiguration, MLObjectType, FIRST_VERSION, LAST_VERSION
import logging

logging.basicConfig(level=logging.ERROR)
#logger = logging.getLogger('pailab.disk_handler')
#logger.setLevel(logging.DEBUG)

tf.VERSION

'1.10.0'

In [2]:
config = {'user': 'test_user',
          'workspace': 'c:/ml_repos/sc',
          'repo_store': 
          {
              'type': 'git_handler',  
              'config': {
                  'folder': 'c:/ml_repos/sc/objects', 
                  'file_format': 'json'
              }
          }
         }
ml_repo = MLRepo(config = config, save_config = True)

In [3]:
repo_store = ml_repo.get_ml_repo_store()

In [4]:
repo_store.commit("")

{'409171f4-fece-11e8-a928-fc084a6691eb', 'e2e67c3e-fec0-11e8-b269-fc084a6691eb', '189d6434-feb3-11e8-9fe7-fc084a6691eb', 'cb9c17f8-fe2a-11e8-9554-fc084a6691eb', '467205d0-fece-11e8-939e-fc084a6691eb', '42ea4552-fece-11e8-84b4-fc084a6691eb', 'cb0f4900-fe2a-11e8-8eb1-fc084a6691eb', 'e6396576-fec0-11e8-b23d-fc084a6691eb', '44754df6-fece-11e8-bc97-fc084a6691eb', '44d83974-fece-11e8-9732-fc084a6691eb', 'cb76e96c-fe2a-11e8-81f4-fc084a6691eb', 'e657dfa2-fec0-11e8-b0d5-fc084a6691eb', '40ca62f4-fece-11e8-a694-fc084a6691eb', 'ea6686c6-fec0-11e8-897c-fc084a6691eb', 'e98a7bb6-fec0-11e8-a3cb-fc084a6691eb', '40f15278-fece-11e8-9649-fc084a6691eb', '1976fdb6-feb3-11e8-8072-fc084a6691eb', '423a3268-fece-11e8-b2bd-fc084a6691eb', '41053be8-fece-11e8-875a-fc084a6691eb', '44eceb38-fece-11e8-b7ee-fc084a6691eb', 'cbcf4a34-fe2a-11e8-872b-fc084a6691eb', '45957ad2-fece-11e8-9123-fc084a6691eb', 'e8ae7092-fec0-11e8-84a7-fc084a6691eb', 'e2ce1978-fec0-11e8-aafd-fc084a6691eb', 'cc058b3e-fe2a-11e8-b7e2-fc084a6691eb',

In [5]:
repo_store.push()

### Setup repo

In [4]:
if True:
    filename_X = 'C:\\Users\\Anwender\\development\\RIVACON\\iqual\\experiments\\sc\\sc_params_600015.npy'
    filename_Y = 'C:\\Users\\Anwender\\development\\RIVACON\\iqual\\experiments\\sc\\prices_600015.npy'  
    ml_repo.raw_data.add_from_numpy_file('sc', filename_X,['initial_variance', 'speed_of_mean_reversion', 
                                                           'long_run_variance', 'vol_of_variance', 'correlation', 'strike', 'expiry'], filename_Y, ['price'] )
    ml_repo.training_data.add('training_small','sc', 0, 10000)
    ml_repo.test_data.add('test_small','sc', 10001, 20000)
    ml_repo.add_measure(MeasureConfiguration.R2)
    ml_repo.add_measure(MeasureConfiguration.MAX)
    
    
    
#the following is for demonstration how to generate data by simulation only
if False: #create new data
    import pyvacon.analytics as analytics
    import pyvacon.tools.converter as converter
    import pyvacon.models.converter as model_converter
    import pyvacon.models.tools as model_tools
    import random
    import math
    simtimes_p = range(1,3*365)
    refdate = analytics.ptime(2017, 1, 1, 0, 0, 0)
    num_threads = 2
    nsims = 10000
    ntimesteps_per_year = 365
    spot = 1.0
    n_strikes_per_sample = 5
    expiry_intervalls =  [[10,365],[366, 2*365], [2*365+1,3*365]]
    n_expiries_per_sample = len(expiry_intervalls)
    n_samples = 40000
    n_overall_samples = n_samples *n_strikes_per_sample*n_expiries_per_sample
    X_result = np.empty((n_overall_samples, 5 + 2))
    Y_result = np.empty((n_overall_samples, 1))
    variance_min = math.log(0.05)
    variance_max = math.log(1.2)
    counter = 0
    for i in range(n_samples):
        if i % 5000 == 0:
            print('sample ' + str(i))
        initial_variance = random.uniform(variance_min, variance_max)
        speed_of_meanreversion = random.uniform(0.1, 5.0)
        long_run_variance = random.uniform(variance_min, variance_max)
        vol_of_variance = random.uniform(0.01, 1.2)
        correlation = random.uniform(-0.9,0.9)
        model = analytics.ScottChesneyModel('t', refdate, spot, initial_variance, speed_of_meanreversion, long_run_variance, vol_of_variance, correlation)
        
        tmp = np.random.random_integers(10,3*365, n_expiries_per_sample)
        simtimes_p = []
        for d in range(n_expiries_per_sample):
            simtimes_p.append( int(np.random.randint(expiry_intervalls[d][0],expiry_intervalls[d][1])) )
        simtimes = converter.createPTimeList(refdate, simtimes_p)
        
        tmp = analytics.ModelLab(model, refdate)
        seed = np.random.randint(1,1024)
        #print('sample ' + str(i) + '  seed ' + str(seed))
        tmp.simulate(simtimes, nsims,ntimesteps_per_year, num_threads, seed)
        for t in range(len(simtimes_p)):
            expiry_index = simtimes_p[t]
            expiry_yf = (expiry_index+1)/365.0
            strikes = np.random.uniform(0.5,1.5, n_strikes_per_sample)
            prices = model_tools.compute_statistics(tmp, strikes, t, 0, lambda x, y: max(x-y,0.0))
            #put_prices = model_tools.compute_statistics(tmp, strikes_puts, expiry_index, 0, lambda x, y: max(y-x,0.0))
            #print(put_prices)
            for k in range(n_strikes_per_sample):
                X_result[counter, 0] = initial_variance
                X_result[counter, 1] = speed_of_meanreversion
                X_result[counter, 2] = long_run_variance
                X_result[counter, 3] = vol_of_variance
                X_result[counter, 4] = correlation
                X_result[counter, 5] = strikes[k]
                X_result[counter, 6] = expiry_yf
                Y_result[counter, 0] = prices[k]; #analytics.calcImpliedVol(call_prices[j], strikes_calls[j], expiry_yf, 1.0, 1.0, 'C')
                counter = counter + 1
        if i % 5000 == 0:
            np.save('C:\\Users\\Anwender\\development\\RIVACON\\iqual\\experiments\\sc\\sc_params_' + str(counter) + '.npy', X_result)
            np.save('C:\\Users\\Anwender\\development\\RIVACON\\iqual\\experiments\\sc\\prices_' + str(counter) + '.npy', Y_result)

    #print(Y_result)
    np.save('C:\\Users\\Anwender\\development\\RIVACON\\iqual\\experiments\\sc\\sc_params.npy', X_result)
    np.save('C:\\Users\\Anwender\\development\\RIVACON\\iqual\\experiments\\sc\\prices.npy', Y_result)


### Add model and train NN

In [3]:


if True:
    from tensorflow.keras import layers
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Dense, Activation

    model = Sequential()
    model.add(Dense(7,input_dim=7))
    model.add(Activation('relu'))
    model.add(Dense(25))
    model.add(Activation('relu'))
    model.add(Dense(15))
    model.add(Activation('relu'))
    model.add(Dense(1))
    model.add(Activation('relu'))
    
    import externals.tensorflow_keras_interface as tf_interface
    tf_interface.add_model(ml_repo, model, 'simple_dense', loss='MSE', epochs=80, batch_size=5000000)
    ml_repo.run_training(run_descendants = True)

In [6]:
import pailab.plot as plot

In [7]:
#?plot.histogram_model_error
plot.histogram_model_error(ml_repo, {ml_repo.models.simple_dense.model(): LAST_VERSION},ml_repo.training_data.training_small())

In [9]:
plot.scatter_model_error(ml_repo,ml_repo.models.simple_dense.model(),ml_repo.test_data.test_small(), 'strike')

AttributeError: 'RepoObjectItem' object has no attribute 'obj'

In [ ]:
#ml_repo.run_evaluation()
for k in MLObjectType:
    names = ml_repo.get_names(k.value)
    for n in names: 
        print(n + '\t  ' + k.value)

In [ ]:
depp = ml_repo.get(ml_repo.models.simple_dense(), version = (FIRST_VERSION, LAST_VERSION))
#ml_repo.models.simple_dense.jobs.eval_job.training_small.load()
#job_info = ml_repo._job_runner.get_info(ml_repo.models.simple_dense.jobs.eval_job.training_small(), ml_repo.models.simple_dense.jobs.eval_job.training_small.obj.repo_info.version)
#print(job_info.trace_back)


In [3]:
import pailab.tools_gc as tools_gc
tools_gc.get_numpy_data(ml_repo.get_numpy_data_store(), 'scottchessny')


{'\\simple_dense\\eval/test_small.hdf5', '\\simple_dense/training_stat.hdf5', '\\simple_dense\\eval/training_small.hdf5', '/sc.hdf5', '\\simple_dense/model.hdf5'}
blob: simple_dense/training_stat.hdf5


In [4]:
from pailab.tools import MLTree

In [5]:
tree = MLTree(ml_repo)
tree.models.simple_dense.jobs.training.load()
tree.models.simple_dense.jobs.training.obj.to_dict()

{'finished': '2018-12-15 16:17:26.510074',
 'model': 'simple_dense',
 'model_param_version': 'last',
 'model_version': 'last',
 'predecessors': [],
 'started': '2018-12-15 16:17:24.343082',
 'state': 'finished',
 'training_data_version': 'last',
 'training_function_version': 'last',
 'training_param_version': 'last',
 'user': 'test_user'}